In [49]:
import pandas as pd
import numpy as np
import cdsapi
import utm
import os 
import json
import sys

In [50]:
df = pd.read_excel('ตารางการสำสวจโรคยางพารา.xlsx',sheet_name='พังงา67')

In [51]:
df = df[1:6]

In [52]:
df

,ลำดับ,ชื่อเกษตรกร,วันที่สำรวจ,ตำบล,อำเภอ,จังหวัด,รหัสแปลง(code),พิกัด,Unnamed: 8,อายุต้นยาง(ปี),พันธุ์ยางพารา,โรคที่พบ,เปอร์เซ็นต์การเกิดโรค,ระดับความรุนแรง
1,29.0,นายบุญเกื้อ วัฒนุกิจ,02/02/67,นาเตย,ท้ายเหมือง,พังงา,67PNA07-1,423961,921346,10,RRIT251,โรคใบร่วงชนิดใหม่,1-10%,1
2,30.0,นายประจิม โสภณ,02/02/67,นาเตย,ท้ายเหมือง,พังงา,67PNA07-2,424051,921255,6,RRIM600,ไม่พบโรคใบร่วงชนิดใหม่,-,-
3,31.0,นายอนัน คำอ่อน,02/02/67,นาเตย,ท้ายเหมือง,พังงา,67PNA07-3,424249,921322,9,RRIM600,โรคใบร่วงชนิดใหม่,11-25%,2
4,32.0,นายกระบวน เสือณรงค์,02/02/67,นาเตย,ท้ายเหมือง,พังงา,67PNA07-4,424315,921264,20,RRIM600,โรคใบร่วงชนิดใหม่,25-50%,3
5,33.0,นางขวัญจิตร หิรัญ,02/02/67,นาเตย,ท้ายเหมือง,พังงา,67PNA07-5,424326,921351,20,"PB235,BPM1",โรคใบร่วงชนิดใหม่,51-75%,4


In [53]:
df = df.iloc[:,[2,3,4,5,7,8]]

In [54]:
df = df[:].rename({'วันที่สำรวจ':'date','พิกัด':'x','Unnamed: 8':'y'},axis=1)

In [55]:
df

,date,ตำบล,อำเภอ,จังหวัด,x,y
1,02/02/67,นาเตย,ท้ายเหมือง,พังงา,423961,921346
2,02/02/67,นาเตย,ท้ายเหมือง,พังงา,424051,921255
3,02/02/67,นาเตย,ท้ายเหมือง,พังงา,424249,921322
4,02/02/67,นาเตย,ท้ายเหมือง,พังงา,424315,921264
5,02/02/67,นาเตย,ท้ายเหมือง,พังงา,424326,921351


In [56]:
def adjust_year_to_gregorian(date_str):
    date_obj = pd.to_datetime(date_str, format='%d/%m/%y')
    gregorian_year = date_obj.year - 43
    # Recreate the date with the adjusted year
    return date_obj.replace(year=gregorian_year).strftime('%Y-%m-%d')

df['date'] = df['date'].apply(adjust_year_to_gregorian)


In [57]:
def utmToLatLon(x:list, y:list, zone:int = 47) -> tuple:
    x, y = np.mean(x), np.mean(y) #find mid point
    return utm.to_latlon(x,y, zone_number=zone, zone_letter='N') #return (lat, lon)

In [58]:
df

,date,ตำบล,อำเภอ,จังหวัด,x,y
1,2024-02-02,นาเตย,ท้ายเหมือง,พังงา,423961,921346
2,2024-02-02,นาเตย,ท้ายเหมือง,พังงา,424051,921255
3,2024-02-02,นาเตย,ท้ายเหมือง,พังงา,424249,921322
4,2024-02-02,นาเตย,ท้ายเหมือง,พังงา,424315,921264
5,2024-02-02,นาเตย,ท้ายเหมือง,พังงา,424326,921351


In [59]:
DIR_OUTPUT = f'../Satun-Phang-Nga/23-1nc-Phang-Nga/'
os.makedirs(DIR_OUTPUT, exist_ok=True)
c = cdsapi.Client()
for i , data in df.iterrows():
    lat, lon = utmToLatLon(x=[data['x']], y=[data['y']], zone=47)
    year = int(data['date'].split('-')[0])
    month = int(data['date'].split('-')[1])
    day = int(data['date'].split('-')[2])
    print(f"\nDownloading {data['date']}-{data['ตำบล']}-{data['อำเภอ']}-{data['จังหวัด']} ({lat}, {lon})")
    c.retrieve(
            'reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis',
                'format': 'netcdf',
                'variable': [
                    '10m_u_component_of_wind', '10m_v_component_of_wind', '2m_dewpoint_temperature',
                    '2m_temperature', 'downward_uv_radiation_at_the_surface', 'evaporation',
                    'soil_temperature_level_1', 'surface_pressure', 'total_precipitation',
                ],
                'year': 2023,
                'month': '12',
                'day': [str(i) for i in range(1, 32)],
                'area' : [lat, lon, lat, lon],                  
                'time': [
                    '00:00', '06:00', '12:00',
                    '18:00',
                ],
            },
            f"{DIR_OUTPUT}{i}-{data['date']}-{data['ตำบล']}-{data['อำเภอ']}-{data['จังหวัด']}.nc")

2024-09-07 19:26:35,297 INFO Welcome to the CDS.
As per our announcements on the Forum, this instance of CDS will be decommissioned on 26 September 2024 and will no longer be accessible from this date onwards.
Please update your cdsapi package to a version >=0.7.2, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
2024-09-07 19:26:35,297 WARNING MOVE TO CDS-Beta
2024-09-07 19:26:35,297 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-09-07 19:26:35,737 INFO Request is completed
2024-09-07 19:26:35,738 INFO Downloading https://download-0008-clone.copernicus-climate.eu/cache-compute-0008/cache/data5/adaptor.mars.internal-1725708163.2970693-576-7-3893e91e-b74a-44a1-be12-d43e62fb7439.nc to ../Satun-Phang-Nga/23-1nc-Phang-Nga/1-2024-02-02-นาเตย-ท้ายเหมือง-พังงา.nc (5.8K)
2024-09-07 19:26:36,897 INFO Download rate 5K/s


2024-09-07 19:26:37,509 INFO Welcome to the CDS.
As per our announcements on the Forum, this instance of CDS will be decommissioned on 26 September 2024 and will no longer be accessible from this date onwards.
Please update your cdsapi package to a version >=0.7.2, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
2024-09-07 19:26:37,510 WARNING MOVE TO CDS-Beta
2024-09-07 19:26:37,510 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-09-07 19:26:37,860 INFO Request is queued
2024-09-07 19:53:17,449 INFO Request is running
2024-09-07 19:55:18,676 INFO Request is completed
2024-09-07 19:55:18,677 INFO Downloading https://download-0005-clone.copernicus-climate.eu/cache-compute-0005/cache/data9/adaptor.mars.internal-1725713675.729764-12689-17-53399190-b38f-4542-9e0e-e959e0119337.nc to ../Satun-Phang-Nga/23-1nc-Phang-Nga/2-2024

2024-09-07 19:55:21,709 INFO Welcome to the CDS.
As per our announcements on the Forum, this instance of CDS will be decommissioned on 26 September 2024 and will no longer be accessible from this date onwards.
Please update your cdsapi package to a version >=0.7.2, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
2024-09-07 19:55:21,710 WARNING MOVE TO CDS-Beta
2024-09-07 19:55:21,711 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-09-07 19:55:22,010 INFO Request is queued
2024-09-07 20:22:06,250 INFO Request is completed
2024-09-07 20:22:06,250 INFO Downloading https://download-0013-clone.copernicus-climate.eu/cache-compute-0013/cache/data5/adaptor.mars.internal-1725715319.3953502-6297-11-68c4a5af-8a64-4d4d-b7ef-5de021a15c8f.nc to ../Satun-Phang-Nga/23-1nc-Phang-Nga/3-2024-02-02-นาเตย-ท้ายเหมือง-พังงา.nc (5.8K)
2024-09-

2024-09-07 20:22:10,307 INFO Welcome to the CDS.
As per our announcements on the Forum, this instance of CDS will be decommissioned on 26 September 2024 and will no longer be accessible from this date onwards.
Please update your cdsapi package to a version >=0.7.2, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
2024-09-07 20:22:10,308 WARNING MOVE TO CDS-Beta
2024-09-07 20:22:10,308 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-09-07 20:22:10,682 INFO Request is queued
2024-09-07 21:45:33,217 INFO Request is running
2024-09-07 21:47:34,950 INFO Request is completed
2024-09-07 21:47:34,951 INFO Downloading https://download-0005-clone.copernicus-climate.eu/cache-compute-0005/cache/data3/adaptor.mars.internal-1725720433.3814592-24542-6-55b1006b-28ec-4ceb-b3cb-3ac914080408.nc to ../Satun-Phang-Nga/23-1nc-Phang-Nga/4-2024

2024-09-07 21:47:38,536 INFO Welcome to the CDS.
As per our announcements on the Forum, this instance of CDS will be decommissioned on 26 September 2024 and will no longer be accessible from this date onwards.
Please update your cdsapi package to a version >=0.7.2, create an account on CDS-Beta and update your .cdsapirc file. We strongly recommend users to check our Guidelines at https://confluence.ecmwf.int/x/uINmFw
2024-09-07 21:47:38,538 WARNING MOVE TO CDS-Beta
2024-09-07 21:47:38,538 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-09-07 21:47:38,893 INFO Request is queued
2024-09-07 22:08:12,380 INFO Request is completed
2024-09-07 22:08:12,381 INFO Downloading https://download-0009-clone.copernicus-climate.eu/cache-compute-0009/cache/data2/adaptor.mars.internal-1725721677.935863-8023-2-2da03b5f-18e4-4a30-a5d3-af468ee846bb.nc to ../Satun-Phang-Nga/23-1nc-Phang-Nga/5-2024-02-02-นาเตย-ท้ายเหมือง-พังงา.nc (5.8K)
2024-09-07